## ETL 
### Extract - Transform - Load

Aquí nos centraremos en realizar los cambios de lo que se descubrio en la EDA, esto incluyendo llenado de valores vacios y repetidos.

In [1]:
#Importamos la libreria que vamos a utilizar para la manipulación de datos.
import pandas as pd

In [6]:
#Para la ETL empezaremos con el dataset de HechosEDA
hechos_df = pd.read_csv('../Data/HechosEDA.csv', encoding= 'ISO-8859-1')
hechos_df.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,DirecciÃ³n Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,1/1/2016,2016,1,1,4:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,1/2/2016,2016,1,2,1:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,1/3/2016,2016,1,3,7:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,1/10/2016,2016,1,10,0:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,SD
4,2016-0005,1,1/21/2016,2016,1,21,5:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÃA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃâA, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


Para este dataset lo unico que se modificará es que los valores vacios de cada columna serán
estandarizados como SD (Sin Dato o 0), para no tener espacios vacios.

Las columnas que contengan valores como SD se dejarán tal cual están ya que es Sin Dato

In [7]:
hechos_df.fillna('SD', inplace= True)

C:\Users\jared\AppData\Local\Temp\ipykernel_22156\3186602049.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'SD' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  hechos_df.fillna('SD', inplace= True)


Con esto, todos nuestros espacios vacios ahora tendran el valor de SD.

En este caso no es necesario separar las fechas ya que ya están separadas, así que con esto terminamos el tratamiento de este dataset y para fines más prácticos solamente transformaremos nuestros datos a un archivo parquet para que sea más liviano. Para poder exportar necesitamos hacer
que la columna de altura sea de tipo string.

In [9]:
hechos_df['Altura'] = hechos_df['Altura'].apply(lambda x: str(x))
hechos_df.to_parquet('../Parquets/Hechos_Post_ETL.parquet')

In [11]:
#Para el data set de victimas, primero importamos los datos
victimas_df = pd.read_csv('../Data/VictimasEDA.csv', encoding= 'ISO-8859-1')
victimas_df.head(10)

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1/1/2016,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,1/1/2016
1,2016-0002,1/2/2016,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,1/2/2016
2,2016-0003,1/3/2016,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,1/3/2016
3,2016-0004,1/10/2016,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,1/21/2016,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2/1/2016
5,2016-0008,1/24/2016,2016,1,24,CONDUCTOR,MOTO,MASCULINO,30,1/24/2016
6,2016-0009,1/24/2016,2016,1,24,PASAJERO_ACOMPAÃANTE,MOTO,MASCULINO,29,1/26/2016
7,2016-0010,1/29/2016,2016,1,29,CONDUCTOR,MOTO,MASCULINO,18,1/29/2016
8,2016-0012,2/8/2016,2016,2,8,CONDUCTOR,MOTO,MASCULINO,22,2/8/2016
9,2016-0013,2/10/2016,2016,2,10,PEATON,PEATON,MASCULINO,16,2/10/2016


Como se comentó en la EDA se conservarán las filas que contengas SD ya que representan gran cantidad de nuestra información. Posteriormente se agregaran dos columnas nuevas que derivan del año y mes de fallecimiento, para poder hacer conteo de las fechas de fallecimiento.

Parece ser que la columna FECHA contiene los mismos datos que la de Fecha_Fallecimiento, pero no como no estamos totalmente seguros de esta información por eso tomaremos ambas columnas como diferente información.

In [12]:
#DD/MM/AA (Formato Actual) Empezamos la extracción de años y meses
fecha_dividida = victimas_df['FECHA_FALLECIMIENTO'].apply(lambda x:x if 'SD' in x else x.split('/'))
años = fecha_dividida.apply(lambda x: x if 'SD' in x else x[2])
meses = fecha_dividida.apply(lambda x: x if 'SD' in x else x[1])

In [13]:
victimas_df['Año_Fallecimiento'] = años
victimas_df['Mes_Fallecimiento'] = meses

In [14]:
victimas_df.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO,Año_Fallecimiento,Mes_Fallecimiento
0,2016-0001,1/1/2016,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,1/1/2016,2016,1
1,2016-0002,1/2/2016,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,1/2/2016,2016,2
2,2016-0003,1/3/2016,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,1/3/2016,2016,3
3,2016-0004,1/10/2016,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD,SD,SD
4,2016-0005,1/21/2016,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2/1/2016,2016,1


Ya tenemos nuestro dataset listo para su utilización en el dashboard, ahora igualmente que el anterior es momento de importarlo a un parquet para tenerlo en un formato más performante.

In [15]:
victimas_df.to_parquet('../Parquets/Victimas_Post_ETL.parquet')